In [12]:
# Step 1: Import required libraries
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Step 2: Load tokenizer and pre-trained DistilBERT model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

# Step 3: Load dataset (IMDB Sentiment Analysis Dataset)
dataset = load_dataset("imdb")

# Step 4: Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Step 5: Convert the dataset to PyTorch format
def format_dataset(dataset):
    return dataset.with_format("torch")

train_dataset = format_dataset(tokenized_datasets["train"])
test_dataset = format_dataset(tokenized_datasets["test"])

# Step 6: Create DataLoaders
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=64)

# Step 7: Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_dataloader) * 3  # 3 epochs
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Step 8: Set up device (GPU/CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Step 9: Training loop
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()  # Set model to training mode

    for batch in tqdm(train_dataloader):
        # Send batch to device
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    print(f"Training loss: {loss.item()}")

# Step 10: Evaluation
model.eval()  # Set model to evaluation mode
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch["labels"].cpu().numpy())

accuracy = accuracy_score(all_labels, all_predictions)
print(f"Test Accuracy: {accuracy}")

# Step 11: Save the fine-tuned model
model.save_pretrained("./fine_tuned_distilbert")
tokenizer.save_pretrained("./fine_tuned_distilbert")

RuntimeError: Failed to import transformers.optimization because of the following error (look up to see its traceback):
cannot import name 'is_torch_hpu_available' from 'transformers.utils' (/opt/anaconda3/envs/FYP/lib/python3.10/site-packages/transformers/utils/__init__.py)